<h1 id="heading">

<a class="anchor-link" href="https://www.kaggle.com/deb009/predict-customer-churn/notebook#heading">¶</a>
</h1>


In [ ]:
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import xgboost as xgb

In [ ]:
!pip install miceforest
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas_profiling import ProfileReport

In [ ]:
PATH = "../input/hackerearths-reduce-marketing-waste/"
train = pd.read_csv(f"{PATH}train.csv",index_col='Deal_title')
test = pd.read_csv(f"{PATH}test.csv",index_col='Deal_title')
submission= pd.read_csv(f"{PATH}sample_submission.csv",index_col='Deal_title')

In [ ]:
test_c=pd.read_csv(f"{PATH}test.csv",index_col='Deal_title')

# 1. Data Exploration

In [ ]:
train.head()

In [ ]:
#adding test and train set together

df =  pd.concat(objs=[train, test], axis=0).reset_index(drop=True)

In [ ]:
df.info()

In [ ]:
profile = ProfileReport(df)
profile

In [ ]:
#We will be removing some of the columns based on the count of unique values
train = train.drop(["Lead_name","Contact_no","POC_name","Lead_POC_email","Date_of_creation"], axis=1)
test = test.drop(["Lead_name","Contact_no","POC_name","Lead_POC_email","Date_of_creation"], axis=1)

In [ ]:
test.isnull().sum()

In [ ]:
train.isnull().sum()

As, there are missing values in train and test we will not be dropping the na values.

In [ ]:
#Two of the columns has dollar sign.We will be removing them now.
colstocheck = train.columns
train[colstocheck] = train[colstocheck].replace({r'\$':''}, regex = True)
train.head()

In [ ]:
colstocheck = test.columns
test[colstocheck] = test[colstocheck].replace({r'\$':''}, regex = True)
test.head()

In [ ]:
#Coverting deal_value and weighted_amount to type numeric

test['Weighted_amount'] = pd.to_numeric(test['Weighted_amount'], errors='coerce')
train['Weighted_amount'] = pd.to_numeric(train['Weighted_amount'], errors='coerce')
train['Deal_value'] = pd.to_numeric(train['Deal_value'], errors='coerce')
test['Deal_value'] = pd.to_numeric(test['Deal_value'], errors='coerce')

In [ ]:
train.info()

# 1.1 Missing_values

In [ ]:
train["Industry"].fillna(train["Industry"].mode()[0], inplace = True)
test["Industry"].fillna(test["Industry"].mode()[0], inplace = True)

In [ ]:
train["Last_lead_update"].fillna(train["Last_lead_update"].mode()[0], inplace = True)
test["Last_lead_update"].fillna(test["Last_lead_update"].mode()[0], inplace = True)

In [ ]:
train["Resource"].fillna(train["Resource"].mode()[0], inplace = True)
test["Resource"].fillna(test["Resource"].mode()[0], inplace = True)

In [ ]:
train['Deal_value'].fillna(train['Deal_value'].median(), inplace=True)
test['Deal_value'].fillna(test['Deal_value'].median(), inplace=True)

In [ ]:
train['Weighted_amount'].fillna(train['Weighted_amount'].median(), inplace=True)
test['Weighted_amount'].fillna(test['Weighted_amount'].median(), inplace=True)

Geography

In [ ]:
train['Geography'].value_counts(dropna=False)

In [ ]:
train[train['Geography'].isnull()]

we can understand from the above data is that the area from USA has a , in between them but areas from India doesn't have them.
So, we can fillup the Gerography with USA where the area has a , and the one which doesn't have a , as India,

In [ ]:
train[['Place', 'State']] = train['Location'].str.split(' ', 1, expand=True)
train['State'].fillna('0', inplace = True)
train['Geography'].fillna('USA', inplace = True)
train.loc[train['State'] == '0', 'Geography'] = 'India'
train.drop(['Place', 'State'], axis = 1, inplace = True)

In [ ]:
test[['Place', 'State']] = train['Location'].str.split(' ', 1, expand=True)
test['State'].fillna('0', inplace = True)
test['Geography'].fillna('USA', inplace = True)
test.loc[test['State'] == '0', 'Geography'] = 'India'
test.drop(['Place', 'State'], axis = 1, inplace = True)

Location

In [ ]:
train[train['Location'].isnull()]

Success_probability

In [ ]:
train['Success_probability'].describe()

In [ ]:
#Probability cannot be more than 100.So, we will be dropping rows having values greater than 100
train.drop(train.loc[train['Success_probability']>100].index, inplace=True)

In [ ]:
train.nunique()

In [ ]:
# Get list of categorical variables
s = (train.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

In [ ]:
from sklearn.preprocessing import LabelEncoder
#coverting the object type columns to cat type
for i in object_cols :
    train[i] = train[i].astype('category')
    test[i] = test[i].astype('category')
#coverting the categorical columns to numeric
# Apply label encoder to each column with categorical data
label_encoder = LabelEncoder()
for col in object_cols:
    train[col] = label_encoder.fit_transform(train[col])
    test[col] = label_encoder.fit_transform(test[col])

In [ ]:
train.head()

In [ ]:
y = train.pop('Success_probability')
X = train

In [ ]:
# split into train dev and test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

In [ ]:
model_xgb = xgb.XGBRegressor(eval_metric='mae')
model_xgb.fit(X_train, y_train, early_stopping_rounds=20, eval_set=[(X_test, y_test)], verbose=1)

In [ ]:
y_pred=model_xgb.predict(X_test)



In [ ]:
from sklearn import metrics
score=max(0,100-np.sqrt(metrics.mean_squared_error(y_test,y_pred)))
print(score)

In [ ]:
test

In [ ]:
final = model_xgb.predict(test)
test['Success_probability']= final
test = test.drop(['Industry','Deal_value','Weighted_amount','Pitch','Lead_revenue','Fund_category','Geography','Location','Designation','Hiring_candidate_role','Lead_source','Level_of_meeting','Last_lead_update','Internal_POC','Resource','Internal_rating'], axis=1)
test.to_csv('output.csv')